In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler,RobustScaler,LabelEncoder,OneHotEncoder
import seaborn as sns

# from helperFunctions import *
# from customPreProc import * 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
def getNullValueCounts(X):
    print(X.isna().sum())

def removeNaN(threshold,X,columns_to_delete,df):
    null_percentage = X.isnull().sum()/len(X)*100
    print('Removed: ')
    # print(null_percentage[null_percentage>threshold])
    # Below code gives list of columns having more than 60% null
    col_to_drop = null_percentage[null_percentage>threshold].keys()
    columns_to_delete.extend(col_to_drop)
    X = X.drop(col_to_drop,axis=1)
    print('old df : '+str(df.shape[1])+" Columns")
    print('new df : '+str(X.shape[1])+" Columns")
    return X,columns_to_delete

def getUnique(X, flag):
    for i in X.columns:
        if flag == True:
            print(i + " : " + str(len(X[i].unique())))
        else:
            print(i + " : " + str(X[i].unique())) 

# remove rows with more than 60% null values 
def removeNullRows (X,t):
    before = X.shape[0]
    X.dropna(thresh=int(t*X.shape[1]),inplace=True)
    after = X.shape[0]
    print("Rows with more than "+str(int(t*X.shape[1]))+" non null values will survive, deleted : "+str(before-after))
    return X

def impute(X,feature,method=None,value=None,done = 0):
    if done == 0:
        print("percentage of null values : "+str(X[feature].isna().sum()/X[feature].shape[0]))
        print("mode : "+str(X[feature].mode()))
        print("mean : "+str(X[feature].mean()))
        print("Value counts : "+str(X[feature].value_counts().head(7)))
        print('\n')

        print(X[feature].describe())
        i = input("want to change (y/n)")
        if i == 'y':
            j = input("custom value or inbuilt method (c/i)")
            if j == 'c':
                val = float(input('value : '))
                X[feature].fillna(val,inplace=True)
            else :
                meth = str(input("Inbuilt method name : "))
                X[feature].fillna(method=meth,inplace=True)
    else:
        if value != None:
            X[feature].fillna(value,inplace=True)
        elif method != None:
            X[feature].fillna(method=method,inplace=True)

def fillNullRows(X):
    try:
        # X['DEF_60_CNT_SOCIAL_CIRCLE'].isna().sum()/X['DEF_60_CNT_SOCIAL_CIRCLE'].shape[0]

        X['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0,inplace=True) # since most values are between 0 and 1
        impute(X,"OBS_60_CNT_SOCIAL_CIRCLE", value = 0, done = 1) # ill fill it with 0 because onlu 0.3 % is null and both mean and mode are 0
        impute(X,"DEF_30_CNT_SOCIAL_CIRCLE",value=0,done=1) # 0 because very less data is missing and mode is 0
        impute(X,"OBS_30_CNT_SOCIAL_CIRCLE",value=1.4,done=1) # data is spread along first few values 0,1,2 
        impute(X,"TOTALAREA_MODE",method="bfill",done=1) # data is very evenly spread
        impute(X, 'EXT_SOURCE_2', value = 0.566010, done = 1)
        impute(X, 'AMT_ANNUITY', value = X['AMT_ANNUITY'].mean(), done = 1)
        impute(X, 'BASEMENTAREA_MEDI', value = 0, done = 1)
        impute(X, 'LANDAREA_MEDI', value = 0, done = 1)
        # X['NAME_TYPE_SUITE'].fillna(value='Unaccompanied', inplace=True)
        X['EMERGENCYSTATE_MODE'].fillna(value='No', inplace = True)
        impute(X, 'EXT_SOURCE_1', value = 0.5025566902901272, done = 1)
        impute(X, 'EXT_SOURCE_3', value = 0.510894, done = 1)
        impute(X, 'APARTMENTS_MEDI', value = 0.0825, done = 1)
        impute(X, 'YEARS_BEGINEXPLUATATION_MEDI', value = 0.9871, done = 1)
        impute(X, 'ELEVATORS_MEDI', value = 0.000000, done = 1)
        impute(X, 'ENTRANCES_MEDI', value = 0.1379, done = 1)
        impute(X, 'FLOORSMAX_MEDI', value = 0.1667, done = 1)
        impute(X, 'NONLIVINGAREA_MEDI', value = 0.0000, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_HOUR', value = 0, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_DAY', value = 0, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_WEEK', value = 0, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_MON', value = 0, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_QRT', value = 0, done = 1)
        impute(X, 'AMT_REQ_CREDIT_BUREAU_YEAR', value = 1.897504, done = 1)
        impute(X, 'LIVINGAREA_MEDI', method = "bfill", done = 1)
        # impute(X, 'OCCUPATION_TYPE', method = "bfill", done = 1)
    except KeyError:
        pass
    cols_to_impute = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAREA_MODE', 'AMT_GOODS_PRICE']
    
    for i in cols_to_impute:
        try:
            impute(X, i, value = X[i].mean(), done = 1)
        except KeyError:
            pass


def dropRows(X):
    try:
        X.drop(X.index[X['TOTALAREA_MODE'].isnull()], inplace = True, axis = 0)
        X.drop(X.index[X['LIVINGAREA_MEDI'].isnull()], inplace = True, axis = 0)
        X.drop(axis = 0, index=X.index[X['DAYS_LAST_PHONE_CHANGE'].isnull()], inplace=True)
        X.drop(axis = 0, index=X.index[X['CNT_FAM_MEMBERS'].isnull()], inplace=True)
    except KeyError:
        pass
    return X
#     return X

def replaceValues(X):
    try:
        X["FLAG_OWN_REALTY"] = X["FLAG_OWN_REALTY"].replace(to_replace = "Y", value = 1)
        X["FLAG_OWN_REALTY"] = X["FLAG_OWN_REALTY"].replace(to_replace = "N", value = 0)
        X["FLAG_OWN_CAR"] = X["FLAG_OWN_CAR"].replace(to_replace = "Y", value = 1)
        X["FLAG_OWN_CAR"] = X["FLAG_OWN_CAR"].replace(to_replace = "N", value = 0)
    except KeyError:
        pass
    return X

def encodeValues(X):
    labelEncoder = LabelEncoder()
    oneHotEncoder = OneHotEncoder()
    try:
        X['EMERGENCYSTATE_MODE'] = labelEncoder.fit_transform(X['EMERGENCYSTATE_MODE'])
        X['NAME_CONTRACT_TYPE'] = labelEncoder.fit_transform(X['NAME_CONTRACT_TYPE'])
        X = pd.get_dummies(X, columns = ['CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'ORGANIZATION_TYPE','OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START', 'WALLSMATERIAL_MODE','HOUSETYPE_MODE'])
    except KeyError:
        pass
    return X

'''Returns columns to drop , cols that are highly correlated'''
def getCorr(X):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
    print("There are "+str(len(to_drop))+" corellated columns :")
    for i in to_drop:
        print(i)
#     columns_to_delete.extend(to_drop)
    return to_drop


def standardization(X,cols,method):
    scaler = None

    if method == "minmax":
        scaler = MinMaxScaler()
    elif method == "stdsc":
        scaler = StandardScaler()
    elif method == "robust":
        scaler = RobustScaler()
    else: 
        return X
    return pd.DataFrame(scaler.fit_transform(X[cols]))

def replaceNegativeColumns(X):
    for i in X:
        X[i] = X[i].abs()
    return X

def getCategoricalColumns(X):
    categorical = []
    numerical = []
    for i in X:
        if X[i].nunique() <= 2:
            categorical.append(i)
        else:
            numerical.append(i)
    return categorical, numerical
def initdf():
    df = pd.read_csv('./train_data.csv')
    X = df.drop(['TARGET'],axis=1)
    y = df['TARGET']
    return df,X,y

def getBoxPlot(X):
    for i in range(0,len(X.columns)-5,6):
        plt.figure(figsize=(18,9))
        X[X.columns[i:i+6]].boxplot()
        plt.title("Numerical variables in Modcloth dataset", fontsize=20)
        plt.show()
def plotBox(X):
    for i in range(0,len(X.columns)-5,6):
        plt.figure(figsize=(18,9))
        X[X.columns[i:i+6]].boxplot()
        plt.title("Numerical variables in Modcloth dataset", fontsize=20)
        plt.show()


def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1

    lower_bracket = Q1-1.5*IQR
    upper_bracket = Q3+1.5*IQR

    lower = df[~(df<lower_bracket)].min()
    upper = df[~(df>upper_bracket)].max()

    arr = np.where(df > upper,np.nan,np.where(df < lower,np.nan, df))
    pd.DataFrame(arr, index=df.index, columns=df.columns)
    return df

In [ ]:
columns_to_delete = []
df = pd.read_csv('./train_data.csv')

def initxy(df):
    X = df.drop(['TARGET'],axis=1)
    y = df['TARGET']
    return X,y
print("Percentage of 1s : " + str((len(df['TARGET'][df['TARGET'] == 1])/len(df['TARGET'])) * 100) + " %")
print("Percentage of 0s : " + str((len(df['TARGET'][df['TARGET'] == 0])/len(df['TARGET'])) * 100) + " %")
X = df
cols_to_delete = ['SK_ID_CURR']
columns_to_delete.extend(cols_to_delete)
X.drop(cols_to_delete,axis=1,inplace=True)

In [ ]:
x_num = X.select_dtypes(exclude=['object'])
pd.DataFrame(x_num).to_csv("numerical.csv", index = False)
categorical_cols = X.select_dtypes(include = ['object'])
numerical_cols = X.select_dtypes(include = ['number'])

In [ ]:
''' Detecting outliers and filling it with NaN '''

# from outliers import remove_outlier_IQR

dfn = pd.read_csv('./numerical.csv')
X_numeric = dfn.drop(['TARGET'],axis = 1 )
y = dfn['TARGET']

X_numeric = remove_outlier_IQR(X_numeric)

print(len(X.columns))
for col in X_numeric.columns:
    X[col] = X_numeric[col]
print(len(X.columns))

# X has no outliers
# Comment above code to contniue using data with outliers



### outlier detection end


In [ ]:
getNullValueCounts(X)

In [ ]:
X,columns_to_delete = removeNaN(65,X, columns_to_delete, df)
type(X)

In [ ]:
X = removeNullRows(X,0.65)
print("new df size : "+str(X.shape))

X = replaceValues(X)

''' Get columns with object type data'''
dtype_object_list = X.select_dtypes(include=['object'])
print('dtype_object_list shape',dtype_object_list.shape)

fillNullRows(X)
X = dropRows(X)
X = encodeValues(X)

In [ ]:
getNullValueCounts(X)

In [ ]:
std_scalar = ["DAYS_BIRTH","DAYS_ID_PUBLISH"] # for gaussian
min_max = ["CNT_CHILDREN","AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY","DAYS_REGISTRATION","CNT_FAM_MEMBERS","REGION_RATING_CLIENT","REGION_RATING_CLIENT_W_CITY","OBS_30_CNT_SOCIAL_CIRCLE","DEF_30_CNT_SOCIAL_CIRCLE","OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE","DAYS_LAST_PHONE_CHANGE","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_YEAR"] #for non gaussian
robust = ["REGION_POPULATION_RELATIVE","DAYS_EMPLOYED"] #for outliers

In [ ]:
getNullValueCounts(X)

In [ ]:
X.drop(X.index[X['AMT_CREDIT'].isnull()], axis = 0, inplace = True)

y = X['TARGET']
X = X.drop(['TARGET'],axis=1)

fillNullRows(X)
getNullValueCounts(X)
X = replaceNegativeColumns(X)
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [ ]:
test_X = pd.read_csv('./test_data.csv')

In [ ]:
test_X = replaceValues(test_X)
fillNullRows(test_X)
test_X = encodeValues(test_X)
impute(test_X, 'CNT_FAM_MEMBERS', method = 'bfill', done = 1)
impute(test_X, 'LIVINGAREA_MEDI', value = 0.109127, done = 1)
impute(test_X, 'TOTALAREA_MODE', value = 0.103063, done = 1)
cols_to_impute = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAREA_MODE']
for i in cols_to_impute:
    impute(test_X, i, value = test_X[i].mean(), done = 1)
test_X_ID = test_X['SK_ID_CURR']
columns_to_delete.append('NAME_FAMILY_STATUS_Unknown')
test_X.drop(columns_to_delete, axis = 1, inplace=True)
test_X = replaceNegativeColumns(test_X)
getNullValueCounts(test_X)
test_X = test_X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train,y_train)

In [ ]:
new_X_cols = []
lgbFeatureSelection = pd.Series(clf.feature_importances_)
lgbFeatureSelection.index = X.columns
new_X_cols.extend(lgbFeatureSelection[lgbFeatureSelection > 20].index)
lgbFeatureSelection.sort_values(ascending=False)

In [ ]:
no_of_ones = len(y_train[y_train == 1])
no_of_zeros = len(y_train[y_train == 0])
clf = lgb.LGBMClassifier(scale_pos_weight=(9.1985/2))
clf.fit(X_train[new_X_cols],y_train)
y_pred = clf.predict(X_test[new_X_cols])
print("Training(Validation) Accuracy: " + str(np.mean(y_pred == y_test) * 100))
y_pred=clf.predict(test_X[new_X_cols])
# len(X_test)
soln_df = pd.DataFrame({"SK_ID_CURR" : test_X_ID, "TARGET" : y_pred})
print(soln_df.value_counts(["TARGET"]))
soln_df.to_csv("solution.csv", index = False)